## Pytorch를 이해하기 위해 간단한 MNIST Classification코드를 먼저 작성하고, 뒤에서 GAN을 코딩

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
# pytorch에서는 'cpu'로 신경망을 동작할지, 'cuda'로 할지, 'tpu'로 할지 디바이스 선택
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
# 신경망 구성시에 난수를 발생시키는 seed value로 123을 설정
random.seed(123)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
# Hyper-Parameter
learning_rate = 0.001
train_epochs = 15
batch_size = 128

In [5]:
# MNIST dataset 가져오기
# train
mnist_train = dsets.MNIST(root='MNIST_data/', 
                          train=True, 
                          transform=transforms.ToTensor(), 
                          download=True)

# test
mnist_test = dsets.MNIST(root='MNIST_data/', 
                         train=False, 
                         transform=transforms.ToTensor(),
                         download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [6]:
# pytorch는 DataLoader를 이용해서 데이터를 공급
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                     batch_size = batch_size,
                                     shuffle = True,
                                     drop_last = True)

In [7]:
# Neural Network 구성
# 784 -> 256 -> 128 -> 10
input_node = 784
hidden1 = 256
hidden2 = 128
output_class = 10

# y= wx + b 
linear1 = torch.nn.Linear(input_node, hidden1, bias=True)
linear2 = torch.nn.Linear(hidden1, hidden2, bias=True)
linear3 = torch.nn.Linear(hidden2, output_class, bias=True)
relu = torch.nn.ReLU()

In [8]:
# Init weights
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[ 0.5687, -0.1612, -0.1584,  ...,  1.1119,  0.7170, -0.6875],
        [ 0.6073,  0.9412,  1.7536,  ..., -0.9421, -0.0112,  2.2780],
        [ 0.6959,  0.7964,  0.9020,  ..., -1.2077, -1.5241, -0.9220],
        ...,
        [-0.7727, -2.1631, -1.9006,  ..., -0.4046, -0.6868,  2.0359],
        [-1.6173, -0.5779,  0.2445,  ...,  0.8612,  1.1035, -0.1492],
        [ 1.9185, -0.2336,  0.5423,  ..., -2.6938, -1.3403, -0.0556]],
       requires_grad=True)

In [9]:
# 앞에서 정의했던 신경망을 to(device)에 구성해라.
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [10]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [11]:
total_batch = len(data_loader)
for epoch in range(train_epochs):
    avg_cost = 0

    for X, Y in data_loader: #batch_size만큼씩 X,Y를 가져온다.
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 116.366775513
Epoch: 0002 cost = 29.625452042
Epoch: 0003 cost = 18.511127472
Epoch: 0004 cost = 13.026321411
Epoch: 0005 cost = 9.708135605
Epoch: 0006 cost = 7.383376598
Epoch: 0007 cost = 5.767514229
Epoch: 0008 cost = 4.541966915
Epoch: 0009 cost = 3.548433781
Epoch: 0010 cost = 2.734172821
Epoch: 0011 cost = 2.134626389
Epoch: 0012 cost = 1.668107390
Epoch: 0013 cost = 1.262706518
Epoch: 0014 cost = 1.005661011
Epoch: 0015 cost = 0.770340681
Learning finished


## GAN을 구성해보자

In [12]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torchvision.utils import save_image 

In [13]:
# pytorch에서는 'cpu'로 신경망을 동작할지, 'cuda'로 할지, 'tpu'로 할지 디바이스 선택
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [14]:
# GAN을 위한 Hyper-parameter
latent_size = 64
hidden_size = 256
image_size  = 784 # MNIST
num_epochs  = 200

# MNIST train 60000 /100 = 600
batch_size  = 100

# 생성된 이미지를 저장
sample_dir = 'samples' 

In [15]:
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [16]:
# 이미지 데이터셋 전처리(ToTensor(), Normalize())
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5], std=[0.5]) # 0~ 1사이의 pixel값으로 
                                ])

In [17]:
mnist_train = dsets.MNIST(root='MNIST_data/', 
                          train=True, 
                          transform=transform, 
                          download=True)

In [18]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                     batch_size = batch_size,
                                     shuffle = True,
                                     drop_last = True)

In [19]:
from torch.nn.modules.activation import LeakyReLU
# GAN의 Discriminator, Generator를 구성
# 784 -> 256 -> 256 -> 1(sigmoid) -> binary classification
# Discriminator
# real, fake
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2), # LeakyReLU의 0미만 부분의 기울기 0.2로 설정
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()) 

In [20]:
# Generator
# 64(latent_space) -> 256 -> 256 -> 784 -> fake image
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [21]:
# D, G는 cuda에...
D = D.to(device)
G = G.to(device)

In [22]:
learning_rate = 0.0002

# Loss(Object Func) + optimizer
criterion = nn.BCELoss()

d_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate)
g_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate)

In [23]:
# 생성된 이미지를 시각화하려면 0~1사이의 픽셀값을 0~255사이로 변환
def denorm(x):
    out = (x+1) / 2
    return out.clamp(0, 1) 

In [24]:
# 학습 코드 구성
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):

        # STEP 1. 이미지를 batch_size만큼 가져와서 1차원으로 변경해서 GPU에 저장
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        # 실제 이미지는 레이블 1, 가짜 이미지는 레이블 0
        # Discreminator를 위해 레이블을 생성
        # STEP 2. 레이블 생성
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1

        # STEP 3. 실제 이미지를 판별기에 넣어서 결과를 얻어내고 이에 대한 loss값을 구한다.
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0

        # STEP 4. 가짜 이미지를 생성하고, 가짜 이미지를  판별기에 넣어서 loss값을 구한다.
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        g_optimizer.zero_grad()

        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

Epoch [0/200], Step [200/600], d_loss: 0.0672, g_loss: 3.6506, D(x): 0.99, D(G(z)): 0.06
Epoch [0/200], Step [400/600], d_loss: 0.0545, g_loss: 5.7537, D(x): 0.98, D(G(z)): 0.03
Epoch [0/200], Step [600/600], d_loss: 0.0505, g_loss: 4.9814, D(x): 0.99, D(G(z)): 0.04
Epoch [1/200], Step [200/600], d_loss: 0.0314, g_loss: 5.0692, D(x): 0.99, D(G(z)): 0.02
Epoch [1/200], Step [400/600], d_loss: 0.1472, g_loss: 4.6531, D(x): 0.92, D(G(z)): 0.03
Epoch [1/200], Step [600/600], d_loss: 0.1393, g_loss: 5.6032, D(x): 0.95, D(G(z)): 0.06
Epoch [2/200], Step [200/600], d_loss: 0.2504, g_loss: 3.8283, D(x): 0.92, D(G(z)): 0.10
Epoch [2/200], Step [400/600], d_loss: 0.3462, g_loss: 4.1661, D(x): 0.85, D(G(z)): 0.07
Epoch [2/200], Step [600/600], d_loss: 0.3396, g_loss: 3.3057, D(x): 0.90, D(G(z)): 0.14
Epoch [3/200], Step [200/600], d_loss: 1.1853, g_loss: 2.0712, D(x): 0.66, D(G(z)): 0.33
Epoch [3/200], Step [400/600], d_loss: 0.4640, g_loss: 3.7411, D(x): 0.87, D(G(z)): 0.20
Epoch [3/200], Step [

In [27]:
!pwd

/content


In [25]:
save_path ='/content/drive/MyDrive/GAN/gan_samples'
!mkdir {save_path}

In [26]:
!cp -r /content/samples/* {save_path}